In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import torch
import lpips
from torchvision import transforms
from dataloaders import Kodak
from PIL import Image
import yaml

In [49]:
lpips_loss = lpips.LPIPS(net='alex') 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/eric/miniconda3/envs/control2/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/eric/miniconda3/envs/control2/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/eric/miniconda3/envs/control2/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [18]:
def ntc_preprocess(image):
    # transform = transforms.Compose(
    #         [transforms.Grayscale(), transforms.ToTensor()]
    #     )
    transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )
    image = transform(image)
    return image

In [19]:
dm = Kodak(root='/home/Shared/image_datasets/Kodak/', batch_size=1)
dset = dm.test_dset

In [20]:
dset[1][0].shape

torch.Size([3, 768, 512])

In [58]:
name = 'Kodak_recon'
exp = 'SD_pi+hed'

loss = 0
bpp = 0
for i, im in enumerate(dset):
    recon = Image.open(f'recon_examples/{exp}/{name}/{i}_recon_0.png')
    recon = ntc_preprocess(recon)
    orig = transforms.functional.to_pil_image(im[0])
    orig = ntc_preprocess(orig)
    loss += lpips_loss(orig.unsqueeze(0), recon.unsqueeze(0)).item()

    with open(f'recon_examples/{exp}/{name}/{i}_caption.yaml', 'r') as stream:
        f = yaml.safe_load(stream)
    bpp += f['bpp_total']
print(f'Avg LPIPS={loss/len(dset)}, bpp={bpp/len(dset)}')

Avg LPIPS=0.664990060031414, bpp=0.035863028632269964


In [ ]:
# q2_ms-ssim: bpp=0.05042, LPIPS=0.51049
# q1_ms-ssim: bpp=0.03066, LPIPS=0.5927511
# q1_mse : bpp=0.01841, LPIPS=0.62766

In [66]:
lpips_loss(orig.repeat(3, 1, 1, 1), recon.repeat(3, 1, 1, 1)).squeeze()

tensor([0.5878, 0.5878, 0.5878], grad_fn=<SqueezeBackward0>)